In [1]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaster2df
from beta_nlp.eval_method import CrossValidation,TrainDevTest
from beta_nlp.experiment import Experiment
from beta_nlp.metrics import Accuracy,Precision,Recall,F1Score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from beta_nlp.preprocess import BasicTextPreprocess,BasicTextFeatureExtraction

In [2]:
# import sys
# sys.path.append("../")
# from beta_nlp.utils.data_util import biocaser2text
# data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
# data_df = biocaser2text(data_file)
# data_df["source"] = "Biocaster"
# from sklearn.utils import shuffle
# data_df = shuffle(data_df).reset_index()
# data_df["flag"]=None
# train_index = int(len(data_df.index)*0.8)
# valid_index = train_index + round(len(data_df.index)*0.1)
# data_df.iloc[:train_index]["flag"]="train"
# data_df.iloc[train_index:valid_index]["flag"]="valid"
# data_df.iloc[valid_index:]["flag"]="test"
# data_df.head(3)

In [3]:
import pandas as pd
data_df = pd.read_csv("../datasets/biocls_26k.csv")
data_df.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,32,India's Deccan gets more planes\n \n \n \n Air...,0,bbc,train
1,1,10431,\n \n DECLINES COMMENT ON HONGKONG LAND REPOR...,0,Reuters,train
2,2,1676,Letter to XPO staff reveals extra safety measu...,1,promed_extended,train
3,3,13747,\n \n \r\n \n KEYCORP <KEY> 3RD QTR NET\r\n \n...,0,Reuters,train
4,4,4435,\n \n \r\n \n API SAYS DISTILLATE STOCKS OFF 2...,0,Reuters,train


In [4]:
bp = BasicTextPreprocess()

# bfe = BasicTextFeatureExtraction(["tfidf","bow","bigram",["tfidf","bigram"],["bow","bigram"],["tfidf","trigram"],["bow","trigram"],["tfidf","bigram","trigram"]])
bfe = BasicTextFeatureExtraction(["tfidf"])


cv = CrossValidation(n_splits=10, n_repeats=10, random_state=12345)
tdt = TrainDevTest()

acc = Accuracy()
pre = Precision()
rec = Recall()
f1 = F1Score()

svm = LinearSVC(random_state=0, tol=1e-5,max_iter=5000)
svm.name = "svm"
dtree = DecisionTreeClassifier(random_state=0,criterion='entropy')
dtree.name="dtree"
bnb = BernoulliNB()
bnb.name = "BernoulliNB"
gnb = GaussianNB()
gnb.name = "GaussianNB"
mnb = MultinomialNB()
mnb.name = "MultinomialNB"
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.name = "RandomForest"

# chi2 feature selection
ch2 = SelectKBest(chi2, k=9000)
svm_ch2 = Pipeline([('sel', ch2), ('cls', svm)])
svm_ch2.name = "svm+chi2"

# chi2 feature selection
dtree_ch2 = Pipeline([('sel', ch2), ('cls', dtree)])
dtree_ch2.name = "dtree+chi2"

# chi2 feature selection
bnb_ch2 = Pipeline([('sel', ch2), ('cls', bnb)])
bnb_ch2.name = "BernoulliNB+chi2"

# chi2 feature selection
gnb_ch2 = Pipeline([('sel', ch2), ('cls', gnb)])
gnb_ch2.name = "GaussianNB+chi2"

# chi2 feature selection
mnb_ch2 = Pipeline([('sel', ch2), ('cls', mnb)])
mnb_ch2.name = "MultinomialNB+chi2"

# chi2 feature selection
mnb_rf = Pipeline([('sel', ch2), ('cls', rf)])
mnb_rf.name = "RandomForest+chi2"

In [6]:
exp = Experiment(
    data_df = data_df,
    preprocessor = bp,
    extractor = bfe,
    eval_method=tdt,
    models=[svm,dtree,bnb,gnb,mnb,svm_ch2,dtree_ch2,bnb_ch2,gnb_ch2,mnb_ch2],
#     models = [svm],
    metrics=[acc,pre,rec,f1],
    result_file = "../results/traditional_bio26k_result.csv",
    verbose = True
).run()

clean_html...
remove_spaces...
remove_punctuation...
stemming...
lemmatization...
Found feature_columns: ['tfidf']
Experiment with svm model on tfidf feature.
../results/traditional_bio26k_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s)       dataset  \
0  0.989599   0.963636  0.966146  0.964889  15.539331  0.943404  Default_name   

  model feature  
0   svm   tfidf  
Experiment with dtree model on tfidf feature.
../results/traditional_bio26k_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s)       dataset  \
0  0.965331   0.895161  0.867188  0.880952   481.4154  1.012973  Default_name   

   model feature  
0  dtree   tfidf  
Experiment with BernoulliNB model on tfidf feature.
../results/traditional_bio26k_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Tr